In [1]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
import sys
import json
import time
import joblib
import pandas as pd
import numpy as np

In [2]:

# Setting up the Python consumer
bootstrap_servers = ['localhost:9092']
input_topic = 'Topic_A'
output_topic = 'Topic_B'  # Topic for output
#group_id = 'my_group_id1'
consumer = KafkaConsumer(input_topic, bootstrap_servers=bootstrap_servers,
                         auto_offset_reset='earliest', value_deserializer=lambda x: json.loads(x.decode('utf-8')))

# Setting up the Kafka producer for output
#ml_producer = KafkaProducer(bootstrap_servers=bootstrap_servers)
ml_producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [3]:
# Load the Time Series Random Forest classification model
rf_model = joblib.load('TS_random_forest_model.joblib')  

In [4]:
window_size = 16
features = ['RR', 'SPO2', 'MAP', 'SBP', 'DBP', 'HR', 'PP', 'CO']

In [5]:
# Update the number of features based on your training data
num_features = 8

# Buffer to maintain the most recent observations
window_buffer = np.zeros((window_size, num_features))

# Reading messages from the input topic and producing results to the output topic
try:
    for message in consumer:
        # Extract features based on your input data
        input_data = message.value
        selected_features = [
            input_data.get("RR", ""),
            input_data.get("SPO2", ""),
            input_data.get("MAP", ""),
            input_data.get("SBP", ""),
            input_data.get("DBP", ""),
            input_data.get("HR", ""),
            input_data.get("PP", ""),
            input_data.get("CO", "")
        ]

        # Update circular buffer with the new data point
        window_buffer[:-1] = window_buffer[1:]
        window_buffer[-1] = selected_features

        # Create a DataFrame with the window buffer
        features_df = pd.DataFrame(window_buffer, columns=['RR', 'SPO2', 'MAP', 'SBP', 'DBP', 'HR', 'PP', 'CO'])

        # Make predictions using the Random Forest classification model
        predicted_class = rf_model.predict(features_df)

        # Create the result dictionary
        prediction_result = {
            "Predicted_Class": predicted_class[0]
        }

        # Concatenate input_data and prediction_result
        output_data = {**input_data, **prediction_result}

        print(f"Prediction result: {output_data}")

        # Produce the result to the output topic
        ml_producer.send(output_topic, value=output_data)

except KeyboardInterrupt:
    pass


Prediction result: {'Timestamp': '2020-10-24 21:37', 'RR': '12.1', 'SPO2': '94.3', 'MAP': '64.7', 'SBP': '93', 'DBP': '51.2', 'HR': '90.6', 'PP': '41.8', 'CO': '3787.08', 'Predicted_Class': 'Normal'}
Prediction result: {'Timestamp': '2020-10-24 21:38', 'RR': '11.8', 'SPO2': '94.1', 'MAP': '61.7', 'SBP': '88.2', 'DBP': '49.2', 'HR': '88.1', 'PP': '39', 'CO': '3435.9', 'Predicted_Class': 'Normal'}
Prediction result: {'Timestamp': '2020-10-24 21:39', 'RR': '12.2', 'SPO2': '94', 'MAP': '62.9', 'SBP': '89.6', 'DBP': '49.8', 'HR': '89.2', 'PP': '39.8', 'CO': '3550.16', 'Predicted_Class': 'Normal'}
Prediction result: {'Timestamp': '2020-10-24 21:40', 'RR': '13.2', 'SPO2': '93.7', 'MAP': '63.7', 'SBP': '90.8', 'DBP': '50.6', 'HR': '89.6', 'PP': '40.2', 'CO': '3601.92', 'Predicted_Class': 'Normal'}
Prediction result: {'Timestamp': '2020-10-24 21:41', 'RR': '12.2', 'SPO2': '93.7', 'MAP': '64', 'SBP': '91.8', 'DBP': '50.8', 'HR': '88.7', 'PP': '41', 'CO': '3636.7', 'Predicted_Class': 'Normal'}
Pr

In [6]:
display(features_df)

,RR,SPO2,MAP,SBP,DBP,HR,PP,CO
0,13.9,96.4,66.5,88.0,55.6,106.1,32.4,3437.64
1,14.7,96.0,65.8,86.5,55.2,105.7,31.3,3308.41
2,15.4,96.0,74.5,97.1,63.5,106.4,33.6,3575.04
3,16.1,96.8,70.0,91.9,58.8,106.5,33.1,3525.15
4,19.2,96.2,63.4,85.0,52.1,105.2,32.9,3461.08
5,19.5,96.3,60.7,0.0,0.0,104.5,0.0,0.00
6,18.9,96.4,63.8,84.4,52.9,105.4,31.5,3320.10
7,19.1,96.8,62.3,83.7,51.7,105.2,32.0,3366.40
8,17.7,96.2,63.4,83.9,52.6,106.0,31.3,3317.80
9,13.2,96.2,62.4,82.9,51.8,105.0,31.1,3265.50
